In [ ]:
#Failide lugemiseks Colabis
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/AI/semantic-diffusion-model-main")

Mounted at /content/drive


In [ ]:
!pip install lpips

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.5 MB/s eta 0:00:00


In [23]:
from traitlets.traitlets import default
"""
StarGAN v2
Copyright (c) 2020-present NAVER Corp.

This work is licensed under the Creative Commons Attribution-NonCommercial
4.0 International License. To view a copy of this license, visit
http://creativecommons.org/licenses/by-nc/4.0/ or send a letter to
Creative Commons, PO Box 1866, Mountain View, CA 94042, USA.
"""

import os
import torch
import argparse
import torch.nn as nn
from torchvision import models
from evaluations.default_dataset import get_eval_loader


def normalize(x, eps=1e-10):
    return x * torch.rsqrt(torch.sum(x**2, dim=1, keepdim=True) + eps)


class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = models.alexnet(pretrained=True).features
        self.channels = []
        for layer in self.layers:
            if isinstance(layer, nn.Conv2d):
                self.channels.append(layer.out_channels)

    def forward(self, x):
        fmaps = []
        for layer in self.layers:
            x = layer(x)
            if isinstance(layer, nn.ReLU):
                fmaps.append(x)
        return fmaps


class Conv1x1(nn.Module):
    def __init__(self, in_channels, out_channels=1):
        super().__init__()
        self.main = nn.Sequential(
            nn.Dropout(0.5),
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias=False))

    def forward(self, x):
        return self.main(x)


class LPIPS(nn.Module):
    def __init__(self):
        super().__init__()
        self.alexnet = AlexNet()
        self.lpips_weights = nn.ModuleList()
        for channels in self.alexnet.channels:
            self.lpips_weights.append(Conv1x1(channels, 1))
        self._load_lpips_weights()
        # imagenet normalization for range [-1, 1]
        self.mu = torch.tensor([-0.03, -0.088, -0.188]).view(1, 3, 1, 1).cuda()
        self.sigma = torch.tensor([0.458, 0.448, 0.450]).view(1, 3, 1, 1).cuda()

    def _load_lpips_weights(self):
        own_state_dict = self.state_dict()
        if torch.cuda.is_available():
            state_dict = torch.load('metrics/lpips_weights.ckpt')
        else:
            state_dict = torch.load('metrics/lpips_weights.ckpt',
                                    map_location=torch.device('cpu'))
        for name, param in state_dict.items():
            if name in own_state_dict:
                own_state_dict[name].copy_(param)

    def forward(self, x, y):
        x = (x - self.mu) / self.sigma
        y = (y - self.mu) / self.sigma
        x_fmaps = self.alexnet(x)
        y_fmaps = self.alexnet(y)
        lpips_value = 0
        for x_fmap, y_fmap, conv1x1 in zip(x_fmaps, y_fmaps, self.lpips_weights):
            x_fmap = normalize(x_fmap)
            y_fmap = normalize(y_fmap)
            lpips_value += torch.mean(conv1x1((x_fmap - y_fmap)**2))
        return lpips_value


@torch.no_grad()
def calculate_lpips_given_images(group_of_images):
    # group_of_images = [torch.randn(N, C, H, W) for _ in range(10)]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    lpips = LPIPS().eval().to(device)
    lpips_values = []
    num_rand_outputs = len(group_of_images)

    # calculate the average of pairwise distances among all random outputs
    for i in range(num_rand_outputs-1):
        for j in range(i+1, num_rand_outputs):
            lpips_values.append(lpips(group_of_images[i], group_of_images[j]))
    lpips_value = torch.mean(torch.stack(lpips_values, dim=0))
    return lpips_value.item()


@torch.no_grad()
def calculate_lpips_given_paths(root_path, img_size=256, batch_size=50, test_list=None):
    print('Calculating LPIPS given root path %s' % root_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    paths = [os.path.join(root_path, path) for path in os.listdir(root_path) if path.startswith('samples')]
    loaders = [get_eval_loader(path, img_size, batch_size, shuffle=False, drop_last=False, test_list=test_list) for path in paths]
    loaders_iter = [iter(loader) for loader in loaders]
    num_clips = len(loaders[0])
    print(paths, num_clips)

    lpips_values = []

    for i in range(num_clips):
        group_of_images = [loader.next().cuda() for loader in loaders_iter]
        lpips_values.append(calculate_lpips_given_images(group_of_images))

    lpips_values = torch.tensor(lpips_values)
    print(lpips_values)

    lpips_mean = torch.mean(lpips_values)

    return lpips_mean

if __name__ == '__main__':
    # python -m metrics.lpips --paths PATH_REAL PATH_FAKE
    
    parser = argparse.ArgumentParser()
    parser.add_argument('-f')
    parser.add_argument('--paths', type=str, default="images samples")
    parser.add_argument('--root_path', type=str, default="RESULTS/2023-04-14", help='paths to real and fake images')
    parser.add_argument('--img_size', type=int, default=256, help='image resolution')
    parser.add_argument('--batch_size', type=int, default=50, help='batch size to use')
    args = parser.parse_args()
    lpips_value = calculate_lpips_given_paths(args.root_path, args.img_size, args.batch_size)
    print('LPIPS: ', lpips_value)

Calculating LPIPS given root path RESULTS/2023-04-14
Preparing DataLoader for the evaluation phase...


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


['RESULTS/2023-04-14/samples'] 5


AttributeError: ignored

In [4]:
# calculate inception score with Keras
from math import floor
from numpy import ones
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
 
# assumes images have the shape 299x299x3, pixels in [0,255]
def calculate_inception_score(images, n_split=10, eps=1E-16):
 # load inception v3 model
 model = InceptionV3()
 # convert from uint8 to float32
 processed = images.astype('float32')
 # pre-process raw images for inception v3 model
 processed = preprocess_input(processed)
 # predict class probabilities for images
 yhat = model.predict(processed)
 # enumerate splits of images/predictions
 scores = list()
 n_part = floor(images.shape[0] / n_split)
 for i in range(n_split):
  # retrieve p(y|x)
  ix_start, ix_end = i * n_part, i * n_part + n_part
  p_yx = yhat[ix_start:ix_end]
  # calculate p(y)
  p_y = expand_dims(p_yx.mean(axis=0), 0)
  # calculate KL divergence using log probabilities
  kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
  # sum over classes
  sum_kl_d = kl_d.sum(axis=1)
  # average over images
  avg_kl_d = mean(sum_kl_d)
  # undo the log
  is_score = exp(avg_kl_d)
  # store
  scores.append(is_score)
 # average across images
 is_avg, is_std = mean(scores), std(scores)
 return is_avg, is_std
 
# pretend to load images
print('loaded', images.shape)
# calculate inception score
#is_avg, is_std = calculate_inception_score(images)
print('score', is_avg, is_std)

50
loaded (50, 299, 299, 3)
score 1.0 0.0


In [ ]:
import torch
_ = torch.manual_seed(123)
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
lpips = LearnedPerceptualImagePatchSimilarity(net_type='vgg')
# LPIPS needs the images to be in the [-1, 1] range.
img1 = (torch.rand(10, 3, 100, 100) * 2) - 1
img2 = (torch.rand(10, 3, 100, 100) * 2) - 1
lpips(img1, img2)